<a href="https://colab.research.google.com/github/zacharylazzara/tent-detection/blob/main/Tent_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
if [ ! -d sarpol-zahab-tents ]; then
  git clone https://github.com/tofighi/sarpol-zahab-tents.git
  mkdir -p output/localized
fi

Cloning into 'sarpol-zahab-tents'...


In [2]:
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from glob import glob
import os
from mpl_toolkits.axes_grid1 import ImageGrid
from tqdm import tqdm
import pickle
import shutil

SHOW_OUTPUT = False

def load(path):
  dataset = {}
  for path in sorted(glob(path)):
    mask = np.asarray(Image.open(path).convert('1'))
    if np.sum(mask > 0):
      dataset[os.path.basename(path).split(f".", 1)[0]] = mask
  return dataset

def kmeans_localize(mask, k):
  mask = np.argwhere(mask.T == 1)
  centers = None
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(mask)
  y_kmeans = kmeans.predict(mask)
  centers = kmeans.cluster_centers_
  return centers

dataset = load("sarpol-zahab-tents/data/labels/*")

# alternatively we could scan through and take the image with the least white pixels (since we drop any image that has none)
pixels_per_dot = np.sum(dataset["sarpol_057"] == 1)

print(f"Pixels Per Dot: {pixels_per_dot}")

results = {}
for key, value in tqdm(dataset.items()):
  n = np.sum(value == 1) // pixels_per_dot
  X = kmeans_localize(value, n)

  results[key] = {"count":n, "loc":X}

  fig = plt.figure(figsize=(10,10))
  plt.title(f"Image: {key}, Tents: {n}")
  plt.imshow(value, vmin=0, vmax=1)
  plt.scatter(X[:,0], X[:,1], s=30, marker='x', c="green", alpha=0.8, label="Tent")
  plt.legend()
  if SHOW_OUTPUT:
    print(f"\n\n{key} Tents: {n}")
  plt.savefig(f"output/localized/{n}_{key}")
  if SHOW_OUTPUT:
    plt.show()
  else:
    plt.close()

with open("output/tent_counts.pkl", "wb") as output:
  pickle.dump(results, output, pickle.HIGHEST_PROTOCOL)

shutil.make_archive("tent_counts", "tar", "output")
print("\nDone.")

Pixels Per Dot: 29


100%|██████████| 90/90 [01:43<00:00,  1.15s/it]

Done.
